<a href="https://colab.research.google.com/github/juan026/EVA3D/blob/main/ComputerGraphics_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Code used for Computer Graphics Project 
# Setup Everything

!nvidia-smi
!git clone https://github.com/juan026/EVA3D.git

!pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu113

import sys
import torch
pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{pyt_version_str}"
])
!pip install fvcore iopath
!pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
!pip install -r EVA3D/requirements.txt

In [ ]:
# @title Train a new model using deepfashion dataset
!python train_deepfashion.py --batch 8 --chunk 1 --expname train_deepfashion_512x256 --dataset_path datasets/DeepFashion --depth 5 --width 128 --style_dim 128 --renderer_spatial_output_dim 512 256 --input_ch_views 3 --white_bg --r1 300 --voxhuman_name eva3d_deepfashion --random_flip --eikonal_lambda 0.5 --small_aug --iter 1000 --adjust_gamma --gamma_lb 20 --min_surf_lambda 1.5 --deltasdf --gaussian_weighted_sampler --sampler_std 15 --N_samples 14


In [ ]:
# @title Download Models
%cd EVA3D
from download_models import download_pretrained_models
download_pretrained_models()

Register and download SMPL models [here](https://smpl.is.tue.mpg.de/). Put the downloaded models in the folder smpl_models. Only the neutral one is needed. The folder structure should look like

```
./
├── ...
└── smpl_models/
    ├── smpl/
        └── SMPL_NEUTRAL.pkl
```

In [ ]:
# @title Download SMPL NEUTRAL model
!ls

!mkdir smpl_models
%cd smpl_models
!mkdir smpl
%cd smpl
!ls
!gdown 1DTiAXbVnCRopZ-LObAqKnT-9rv-aySc5
!ls

In [ ]:
# @title Download dataset for training
%cd ../..
!ls
!python download_datasets.py

In [ ]:

# @title Start the Genenration!
!python generation_demo.py --batch 1 --chunk 1 --expname 512x256_deepfashion --dataset_path demodataset --depth 5 --width 128 --style_dim 128 --renderer_spatial_output_dim 512 256 --input_ch_views 3 --white_bg --voxhuman_name eva3d_deepfashion --deltasdf --N_samples 28 --ckpt 420000 --identities 1 --truncation_ratio 0.5
from IPython.display import Image
Image('evaluations/512x256_deepfashion/iter_0420000/random_angles/images_paper_fig/0000000.png')

In [ ]:
# @title Define images for evaluation
import os
# define images for evaluation
root_dataset = "/content/EVA3D/datasets/DeepFashion/images/"

testing_images= ["MEN-Jackets_Vests-id_00005346-01_4_full.png",
                 "WOMEN-Dresses-id_00006993-03_7_additional.png",
                 "WOMEN-Blouses_Shirts-id_00001722-07_7_additional.png"]


img_testing1 = os.path.join(root_dataset, testing_images[0])
img_testing2 = os.path.join(root_dataset, testing_images[1])
img_testing3 = os.path.join(root_dataset, testing_images[2])

In [ ]:
# @title Define code for transformation


# Run transformations
import cv2
import numpy as np

from google.colab.patches import cv2_imshow





In [ ]:
# @title Transformation: Translation
# Translation
image = cv2.imread(img_testing1)
ancho = image.shape[1] #columnas
alto = image.shape[0] # filas

M = np.float32([[1, 0,200],
                [0, 1,150]])
imageOut = cv2.warpAffine(image,M,(ancho,alto))
print('Input Image')
cv2_imshow(image)
print('Output Image')
cv2_imshow(imageOut)
cv2.waitKey(0)
output_img_path = img_testing1[:-4]+"__translation.png"
print(output_img_path)
cv2.imwrite(output_img_path, imageOut)
cv2.destroyAllWindows()

# save basename image to file for inference
with open("file_test1.txt", "w") as ft1:
  ft1.write(os.path.basename(output_img_path))

In [ ]:
# @title Run inference: Translation
!python generation_demo.py --batch 1 --chunk 1 --expname 512x256_deepfashion --dataset_path /content/EVA3D/datasets/DeepFashion --depth 5 --width 128 --style_dim 128 --renderer_spatial_output_dim 512 256 --input_ch_views 3 --white_bg --voxhuman_name eva3d_deepfashion --deltasdf --N_samples 28 --ckpt 420000 --identities 1 --truncation_ratio 0.5 --testing_list /content/EVA3D/file_test1.txt
from IPython.display import Image
Image('evaluations/512x256_deepfashion/iter_0420000/random_angles/images_paper_fig/0000000.png')

In [ ]:
# @title Transformation: Rotation
# Rotation

image = cv2.imread(img_testing2)
ancho = image.shape[1] #columnas
alto = image.shape[0] # filas
# Rotación
M = cv2.getRotationMatrix2D((ancho//2,alto//2),15,1)
imageOut = cv2.warpAffine(image,M,(ancho,alto))
print('Input Image')
cv2_imshow(image)
print('Output Image')
cv2_imshow(imageOut)

output_img_path = img_testing2[:-4]+"__rotation.png"
print(output_img_path)
cv2.imwrite(output_img_path, imageOut)
cv2.waitKey(0)
cv2.destroyAllWindows()

# save basename image to file for inference
with open("file_test1.txt", "w") as ft1:
  ft1.write(os.path.basename(output_img_path))

In [ ]:
# @title Run inference: Rotation
!python generation_demo.py --batch 1 --chunk 1 --expname 512x256_deepfashion --dataset_path /content/EVA3D/datasets/DeepFashion --depth 5 --width 128 --style_dim 128 --renderer_spatial_output_dim 512 256 --input_ch_views 3 --white_bg --voxhuman_name eva3d_deepfashion --deltasdf --N_samples 28 --ckpt 420000 --identities 1 --truncation_ratio 0.5 --testing_list /content/EVA3D/file_test1.txt
from IPython.display import Image
Image('evaluations/512x256_deepfashion/iter_0420000/random_angles/images_paper_fig/0000000.png')

In [ ]:
# @title Transformation: Scale
image = cv2.imread(img_testing3)
# Scale
imageOut = cv2.resize(image,(600,300), interpolation=cv2.INTER_CUBIC)
print('Input Image')
cv2_imshow(image)
print('Output Image')
cv2_imshow(imageOut)
output_img_path = img_testing3[:-4]+"__scale.png"
print(output_img_path)
cv2.imwrite(output_img_path, imageOut)
cv2.waitKey(0)
cv2.destroyAllWindows()

# save basename image to file for inference
with open("file_test1.txt", "w") as ft1:
  ft1.write(os.path.basename(output_img_path))

In [ ]:
# @title Run inference: Scale
!python generation_demo.py --batch 1 --chunk 1 --expname 512x256_deepfashion --dataset_path /content/EVA3D/datasets/DeepFashion --depth 5 --width 128 --style_dim 128 --renderer_spatial_output_dim 512 256 --input_ch_views 3 --white_bg --voxhuman_name eva3d_deepfashion --deltasdf --N_samples 28 --ckpt 420000 --identities 1 --truncation_ratio 0.5 --testing_list /content/EVA3D/file_test1.txt
from IPython.display import Image
Image('evaluations/512x256_deepfashion/iter_0420000/random_angles/images_paper_fig/0000000.png')